## Deliverable 2

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.0752,-159.3190,84.52,73,75,11.50,broken clouds
1,1,Kenora,CA,49.8167,-94.4337,83.37,44,20,11.50,few clouds
2,2,Khatanga,RU,71.9667,102.5000,50.95,87,100,13.44,light rain
3,3,Tasiilaq,GL,65.6145,-37.6368,48.36,58,20,4.41,few clouds
4,4,Sambava,MG,-14.2667,50.1667,69.58,87,63,9.24,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Khatanga,RU,71.9667,102.5000,50.95,87,100,13.44,light rain
4,4,Sambava,MG,-14.2667,50.1667,69.58,87,63,9.24,light rain
7,7,Half Moon Bay,US,37.4636,-122.4286,79.61,77,100,14.97,overcast clouds
11,11,Rikitea,PF,-23.1203,-134.9692,72.16,76,10,16.13,light rain
12,12,Strezhevoy,RU,60.7333,77.5889,61.56,94,100,8.32,overcast clouds
13,13,Dingle,PH,10.9995,122.6711,76.21,89,100,4.65,overcast clouds
15,15,Cidreira,BR,-30.1811,-50.2056,54.55,77,0,5.75,clear sky
18,18,Provideniya,RU,64.3833,-173.3000,53.35,65,98,3.36,overcast clouds
22,22,Cabra,ES,37.4725,-4.4421,78.85,56,0,6.78,clear sky
24,24,Eyl,SO,7.9803,49.8164,75.87,80,100,21.88,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                7
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Khatanga,RU,71.9667,102.5000,50.95,87,100,13.44,light rain
4,4,Sambava,MG,-14.2667,50.1667,69.58,87,63,9.24,light rain
7,7,Half Moon Bay,US,37.4636,-122.4286,79.61,77,100,14.97,overcast clouds
11,11,Rikitea,PF,-23.1203,-134.9692,72.16,76,10,16.13,light rain
12,12,Strezhevoy,RU,60.7333,77.5889,61.56,94,100,8.32,overcast clouds


In [7]:
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Khatanga,RU,50.95,light rain,71.9667,102.5000,
4,Sambava,MG,69.58,light rain,-14.2667,50.1667,
7,Half Moon Bay,US,79.61,overcast clouds,37.4636,-122.4286,
11,Rikitea,PF,72.16,light rain,-23.1203,-134.9692,
12,Strezhevoy,RU,61.56,overcast clouds,60.7333,77.5889,
13,Dingle,PH,76.21,overcast clouds,10.9995,122.6711,
15,Cidreira,BR,54.55,clear sky,-30.1811,-50.2056,
18,Provideniya,RU,53.35,overcast clouds,64.3833,-173.3000,
22,Cabra,ES,78.85,clear sky,37.4725,-4.4421,
24,Eyl,SO,75.87,overcast clouds,7.9803,49.8164,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Khatanga,RU,50.95,light rain,71.9667,102.5000,"""Khatanga"""
4,Sambava,MG,69.58,light rain,-14.2667,50.1667,Melrose
7,Half Moon Bay,US,79.61,overcast clouds,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
11,Rikitea,PF,72.16,light rain,-23.1203,-134.9692,People ThankYou
12,Strezhevoy,RU,61.56,overcast clouds,60.7333,77.5889,Sibiryachka
13,Dingle,PH,76.21,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
15,Cidreira,BR,54.55,clear sky,-30.1811,-50.2056,Hotel Castelo
18,Provideniya,RU,53.35,overcast clouds,64.3833,-173.3000,
22,Cabra,ES,78.85,clear sky,37.4725,-4.4421,MS Fuente Las Piedras
24,Eyl,SO,75.87,overcast clouds,7.9803,49.8164,Amiin Hotel


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Khatanga,RU,50.95,light rain,71.9667,102.5000,"""Khatanga"""
4,Sambava,MG,69.58,light rain,-14.2667,50.1667,Melrose
7,Half Moon Bay,US,79.61,overcast clouds,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
11,Rikitea,PF,72.16,light rain,-23.1203,-134.9692,People ThankYou
12,Strezhevoy,RU,61.56,overcast clouds,60.7333,77.5889,Sibiryachka
13,Dingle,PH,76.21,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
15,Cidreira,BR,54.55,clear sky,-30.1811,-50.2056,Hotel Castelo
22,Cabra,ES,78.85,clear sky,37.4725,-4.4421,MS Fuente Las Piedras
24,Eyl,SO,75.87,overcast clouds,7.9803,49.8164,Amiin Hotel
25,Huarmey,PE,65.61,broken clouds,-10.0681,-78.1522,Hotel Miramar


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))